In [1]:
import os
import sys
import math
import pickle
import pandas as pd
import numpy as np
from numpy import interp
from scipy.optimize import minimize
from scipy.interpolate import UnivariateSpline
import QuantLib as ql
from datetime import timedelta,datetime

In [32]:
import unittest
import QuantLib as ql
from datetime import datetime

# Helper Functions
def py2ql_date(pydate):
    return ql.Date(pydate.day, pydate.month, pydate.year)

class BSmodel:
    def __init__(self, strike, maturity, OptType):
        if OptType == 'C':
            payoff = ql.PlainVanillaPayoff(ql.Option.Call, strike)
        elif OptType == 'P':
            payoff = ql.PlainVanillaPayoff(ql.Option.Put, strike)
        else:
            raise ValueError("OptType must be 'C' or 'P'")
        
        # European call/put
        exercise = ql.EuropeanExercise(maturity)  
        self.european_option = ql.VanillaOption(payoff, exercise)

    def price(self, valDate, und_price, volatility, risk_free_rate, dividend_rate=0):
        ql.Settings.instance().evaluationDate = valDate
        
        # Market parameters
        und_handle = ql.QuoteHandle(ql.SimpleQuote(und_price))
        flat_ts = ql.YieldTermStructureHandle(
            ql.FlatForward(valDate, ql.QuoteHandle(ql.SimpleQuote(risk_free_rate)), ql.Actual365Fixed())
        )
        dividend_yield = ql.YieldTermStructureHandle(
            ql.FlatForward(valDate, ql.QuoteHandle(ql.SimpleQuote(dividend_rate)), ql.Actual365Fixed())
        )
        flat_vol = ql.BlackVolTermStructureHandle(
            ql.BlackConstantVol(valDate, ql.UnitedStates(ql.UnitedStates.NYSE), ql.QuoteHandle(ql.SimpleQuote(volatility)), ql.Actual365Fixed())
        )

        # Black-Scholes-Merton process
        process = ql.BlackScholesMertonProcess(
            und_handle, dividend_yield, flat_ts, flat_vol
        )
        self.european_option.setPricingEngine(ql.AnalyticEuropeanEngine(process))
        return self.european_option.NPV()


In [35]:
import unittest
from datetime import datetime

class TestBSmodel(unittest.TestCase):
    def test_bs_model_pricing(self):
        # Set test parameters
        strike_price = 100
        maturity_date = py2ql_date(datetime(2024, 12, 31))  # Expiration date
        option_type = 'C'  # Call option
        valuation_date = py2ql_date(datetime(2024, 2, 1))  # Valuation date
        spot_price = 105  # Spot price
        volatility = 0.2  # 20% volatility
        risk_free_rate = 0.05  # 5% risk-free rate
        dividend_yield = 0.02  # 2% dividend yield

        # Create option pricing model
        option = BSmodel(strike_price, maturity_date, option_type)
        option_price = option.price(valuation_date, spot_price, volatility, risk_free_rate, dividend_yield)

        # Set a reasonable price range (adjustable)
        self.assertTrue(5.0 < option_price < 100.0, f"Unexpected option price: {option_price}")

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)




.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK
